In [1]:
import PyQt6.QtCore
import os
os.environ["QT_API"] = "pyqt5"


# Use qt backend for matplotlab to use interactive mne plots
%matplotlib qt

In [2]:
import mne 
import analysis.processing
import matplotlib.pyplot as plt
import pandas as pd
import csv
from mne.time_frequency import csd_morlet, csd_fourier
from mne.time_frequency import tfr_morlet
import matplotlib.patches as patches 
import numpy as np

mne.set_log_level(verbose='WARNING', return_old_level=False, add_frames=None)

In [3]:
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100 # 200 

In [26]:
participant_id = 161

# loading data
raw = analysis.processing.loadData(participant_id)

# preprocessig 
raw = analysis.processing.preprocessing(raw) 

#generate epochs
epochs_1, event_dict = analysis.processing.eventEpochdata(raw)


oddball = epochs_1["visual/image_display/onset/stimulus/non_freq"].average()
control = epochs_1["visual/image_display/onset/stimulus/freq"].average()

#PZ ~ 7Z

evokeds = dict(Oddball=oddball, Control=control)
fig  =  mne.viz.plot_compare_evokeds(evokeds, picks=['7Z'], styles = {"Oddball": {"color" :'red'}, "Control":{"color": 'blue'}})

/Users/erangad/Documents/Research/oddball-task/src/analysis/processing.py:51: RuntimeWarning: Fiducial point nasion not found, assuming identity unknown to head transformation
  raw_filtered.set_montage(montage)


KeyboardInterrupt: 

In [4]:
# read group evokeds

oddball_list = []
control_list = []

freqs = np.arange(2, 30, 2)
n_cycles = 1
decim = 1
average = False

count = 0;

for pnum in (151, 152, 153, 161, 162, 163, 171, 172, 173, 181,\
                182, 183, 191, 192, 193, 201, 202, 203, 211, 212, 213,\
                    221, 222, 223, 231, 232, 233, ):
    raw = analysis.processing.loadData(pnum)

    # preprocessig 
    raw = analysis.processing.preprocessing(raw)
    epochs, event_dict = analysis.processing.eventEpocshByBlocks(raw)

    for bnum in (0, 1, 2):
        block = "b" + str(bnum)

        oddball = epochs[f"oddball & block == '{block}'"]
        control = epochs[f"control & block == '{block}'"]

        oddball_list.append(oddball.average())
        control_list.append(control.average())

        oddball_tfr = tfr_morlet(oddball, freqs,
                                n_cycles=n_cycles, decim=decim,
                                return_itc=False)
        oddball_tfr = oddball_tfr.apply_baseline((-0.2, -0.1), mode= 'ratio')
        
        control_tfr = tfr_morlet(control, freqs,
                                n_cycles=n_cycles, decim=decim,
                                return_itc=False)
        control_tfr = control_tfr.apply_baseline((-0.2, -0.1), mode= 'ratio')

        if count == 0 and bnum == 0:
             
            tfr_times = oddball_tfr.times
            tfr_freqs = oddball_tfr.freqs
            ch_names = oddball_tfr.ch_names
            tfr_info = oddball_tfr.info

            control_tfr_array = np.array([control_tfr.data])
            oddball_tfr_array = np.array([oddball_tfr.data])
        else:
            control_tfr_array = np.append(control_tfr_array, [control_tfr.data], axis = 0)
            oddball_tfr_array = np.append(oddball_tfr_array, [oddball_tfr.data], axis = 0)

    count = count +1 
       
title= "ERP"
roi =['7Z']

evokeds = dict(
    oddball=  oddball_list,
    control= control_list,
)

/Users/erangad/Documents/Research/oddball-task/src/analysis/processing.py:51: RuntimeWarning: Fiducial point nasion not found, assuming identity unknown to head transformation
  raw_filtered.set_montage(montage)
/Users/erangad/Documents/Research/oddball-task/src/analysis/processing.py:51: RuntimeWarning: Fiducial point nasion not found, assuming identity unknown to head transformation
  raw_filtered.set_montage(montage)
/Users/erangad/Documents/Research/oddball-task/src/analysis/processing.py:51: RuntimeWarning: Fiducial point nasion not found, assuming identity unknown to head transformation
  raw_filtered.set_montage(montage)
/Users/erangad/Documents/Research/oddball-task/src/analysis/processing.py:51: RuntimeWarning: Fiducial point nasion not found, assuming identity unknown to head transformation
  raw_filtered.set_montage(montage)
/Users/erangad/Documents/Research/oddball-task/src/analysis/processing.py:51: RuntimeWarning: Fiducial point nasion not found, assuming identity unknown

In [17]:
#save

import json
import pickle

# Create a dictionary
tfr_info = {
    'times': tfr_times.tolist(),
    'freqs': tfr_freqs.tolist(),
    'chans': ch_names,
}

# Save the dictionary to a file
with open('tfr_info.json', 'w') as json_file:
    json.dump(tfr_info, json_file)


np.save('control_tfr_array.npy', control_tfr_array)
np.save('oddball_tfr_array.npy', oddball_tfr_array)


with open('evokeds.pkl', 'wb') as file:
    pickle.dump(evokeds, file)

In [4]:
import json
import numpy as np
import pickle

with open('my_dict.json', 'r') as json_file:
    tfr_info = json.load(json_file)


control_tfr_array = np.load('control_tfr_array.npy')
oddball_tfr_array = np.load('oddball_tfr_array.npy')   


with open('evokeds.pkl', 'rb') as file:
    evokeds = pickle.load(file)


tfr_times = np.array(tfr_info["times"])
tfr_freqs = freqs = np.array(tfr_info["freqs"])   
ch_names =   tfr_info["chans"]

In [41]:
analysis.processing.getERPMontage(evokeds)

In [ ]:
mne.viz.plot_compare_evokeds(
    evokeds,
    picks="eeg",
    ci = 0.95,
    styles = {"oddball": {"color" :'red'}, 
                                        "control":{"color": 'blue'}}, 
    axes="topo",
    sphere='auto',
    ylim = dict(eeg=[-10e-6, 10e-6]) )

[<Figure size 3024x1588 with 63 Axes>]

In [25]:
#front

roi = ['1L', '1R','1Z','1LC', '1LB', '2L', '2R', '1RB', '1RC']
ci = 0.95
invert = False

fig, ax = plt.subplots()
mne.viz.plot_compare_evokeds(evokeds, picks= roi, ci = ci , 
                                styles = {"oddball": {"color" :'red'}, 
                                        "control":{"color": 'blue'}}, 
                                    show =False, axes = ax, combine ='mean' )
if(invert): 
    ax.invert_yaxis()
    
# ax.set_ylabel("V")

ax.set_title("ERP of the frontal channels")

plt.show()


In [100]:
#Occipital

roi = ['10L', '9Z','10R', '9L', '8Z', '9R']
ci = 0.95
invert = False

fig, ax = plt.subplots()
mne.viz.plot_compare_evokeds(evokeds, picks= roi, ci = ci , 
                                styles = {"oddball": {"color" :'red'}, 
                                        "control":{"color": 'blue'}}, 
                                    show =False, axes = ax, combine ='mean' )
if(invert): 
    ax.invert_yaxis()
    
# ax.set_ylabel("V")
ax.set_title("ERP of the occiptal channels")

plt.show()


In [27]:
#Left

roi = ['3LB', '3LC','2LB', '1LA', '2LA', '2LC']
ci = 0.95
invert = False

fig, ax = plt.subplots()
mne.viz.plot_compare_evokeds(evokeds, picks= roi, ci = ci , 
                                styles = {"oddball": {"color" :'red'}, 
                                        "control":{"color": 'blue'}}, 
                                    show =False, axes = ax, combine ='mean' )
if(invert): 
    ax.invert_yaxis()
    
# ax.set_ylabel("V")
ax.set_title("ERP of the left channels")

plt.show()

In [102]:
#Right

roi = ['3RB', '3RC','2RB', '1RA', '2RA', '2RC']
ci = 0.95
invert = False

fig, ax = plt.subplots()
mne.viz.plot_compare_evokeds(evokeds, picks= roi, ci = ci , 
                                styles = {"oddball": {"color" :'red'}, 
                                        "control":{"color": 'blue'}}, 
                                    show =False, axes = ax, combine ='mean' )
if(invert): 
    ax.invert_yaxis()
    
# ax.set_ylabel("V")
ax.set_title("ERP of the right channels")

plt.show()

wilkoxon function

In [8]:
import mne
import numpy as np
from scipy import stats

def compute_wilcoxon_rank_biserial(oddball_list, control_list, roi, tmin=0.3277, tmax=0.5):
    """
    Computes the Wilcoxon signed-rank test and the rank-biserial correlation for given oddball and control evoked responses.
    
    Parameters:
    - oddball_list: List of MNE evoked objects for the oddball condition.
    - control_list: List of MNE evoked objects for the control condition.
    - roi: Region of Interest for selecting channels.
    - tmin: Start of the time window for averaging (in seconds).
    - tmax: End of the time window for averaging (in seconds).
    
    Returns:
    - A dictionary containing Wilcoxon W statistic, p-value, and rank-biserial correlation.
    """
    # Generate difference waves
    diff_waves = [
        mne.combine_evoked([oddball_list[i], control_list[i]], weights=[-1, 1])
        for i in range(len(oddball_list))
    ]

    # Extract the mean values over the specified time window and ROI
    evoked_data = np.array([
        np.mean(e.get_data(picks=roi, tmin=tmin, tmax=tmax)) for e in diff_waves
    ])

    # Perform the Wilcoxon signed-rank test against 0
    result = stats.wilcoxon(evoked_data, zero_method='wilcox', correction=True)
    w_statistic = result.statistic
    pval = result.pvalue

    # Calculate the rank-biserial correlation (effect size)
    n_pos = np.sum(evoked_data > 0)
    n_neg = np.sum(evoked_data < 0)
    n = n_pos + n_neg
    rank_biserial_correlation = (n_pos - n_neg) / n

    # Prepare the result dictionary
    results = {
        'W': w_statistic,
        'p_value': pval,
        'rank_biserial_correlation': rank_biserial_correlation
    }

    # Print the results in a readable format
    print(f'P300 Wilcoxon W= {w_statistic:.2f}, p= {pval:.4f}, rank-biserial correlation (r_rb)= {rank_biserial_correlation:.4f}')
    
    return results


In [16]:
import mne
import numpy as np
from scipy import stats

def compute_paired_wilcoxon_rank_biserial(oddball_list, control_list, roi, tmin=0.3277, tmax=0.5):
    """
    Computes the paired Wilcoxon signed-rank test and the rank-biserial correlation between two conditions.
    
    Parameters:
    - oddball_list: List of MNE evoked objects for the oddball condition.
    - control_list: List of MNE evoked objects for the control condition.
    - roi: Region of Interest for selecting channels.
    - tmin: Start of the time window for averaging (in seconds).
    - tmax: End of the time window for averaging (in seconds).
    
    Returns:
    - A dictionary containing Wilcoxon W statistic, p-value, and rank-biserial correlation.
    """
    # Extract the mean values over the specified time window and ROI for both conditions
    oddball_data = np.array([
        np.mean(e.get_data(picks=roi, tmin=tmin, tmax=tmax)) for e in oddball_list
    ])
    control_data = np.array([
        np.mean(e.get_data(picks=roi, tmin=tmin, tmax=tmax)) for e in control_list
    ])

    # Perform the paired Wilcoxon signed-rank test
    result = stats.wilcoxon(oddball_data, control_data, zero_method='wilcox', correction=True)
    w_statistic = result.statistic
    pval = result.pvalue

    # Calculate the rank-biserial correlation (effect size)
    diff = oddball_data - control_data
    n_pos = np.sum(diff > 0)
    n_neg = np.sum(diff < 0)
    n = n_pos + n_neg
    rank_biserial_correlation = (n_pos - n_neg) / n

    # Prepare the result dictionary
    results = {
        'W': w_statistic,
        'p_value': pval,
        'rank_biserial_correlation': rank_biserial_correlation
    }

    # Print the results in a readable format
    print(f'Paired Wilcoxon W= {w_statistic:.2f}, p= {pval:.4f}, rank-biserial correlation (r_rb)= {rank_biserial_correlation:.4f}')
    
    return results


In [5]:
roi =['7Z']

In [6]:
#plot evokeds at 0.9 CI
analysis.processing.getERP(evokeds, roi = roi, ci = 0.95, invert = True)

In [58]:
#plot evokeds at 0.95 CI
analysis.processing.getERP(evokeds, roi = roi, ci = 0.95, invert = True, vlines = [0.0])

In [13]:
diff_waves = []
for i in range(len(oddball_list)):
    diff_waves.append(mne.combine_evoked([oddball_list[i], control_list[i]], weights=[-1, 1]))

In [14]:
import numpy as np
evoked_data = np.array([ [np.mean(np.mean(e.get_data(picks=roi, tmin= 0.3277, tmax= 0.5), axis=1), axis=0)] for e in diff_waves])
from scipy import stats 

result = stats.ttest_1samp(evoked_data, 0)
t =  result.statistic
print(result)
pval = result.pvalue
# df = result.df
# print('t( df = ', str(round(df[0], 2)) ,')= ', str(round(t[0], 2)), 'p = ', str(round(pval[0], 4)))
print('P300 t( df = 81)= ', str(round(t[0], 2)), 'p = ', str(round(pval[0], 4)))

Ttest_1sampResult(statistic=array([-9.53950155]), pvalue=array([7.6738854e-15]))
P300 t( df = 81)=  -9.54 p =  0.0


In [17]:
# wilkoxon 
compute_paired_wilcoxon_rank_biserial(oddball_list, control_list, roi, tmin=0.3277, tmax=0.5)

Paired Wilcoxon W= 244.00, p= 0.0000, rank-biserial correlation (r_rb)= 0.6790


{'W': 244.0,
 'p_value': 2.612166677600553e-11,
 'rank_biserial_correlation': 0.6790123456790124}

In [71]:
import numpy as np
evoked_data = np.array([ [np.mean(np.mean(e.get_data(picks=roi, tmin=.2772, tmax=.3202), axis=1), axis=0)] for e in diff_waves])
from scipy import stats 

result = stats.ttest_1samp(evoked_data, 0)
t =  result.statistic
print(result)
pval = result.pvalue
# df = result.df
# print('t( df = ', str(round(df[0], 2)) ,')= ', str(round(t[0], 2)), 'p = ', str(round(pval[0], 4)))
print(' N200 t( df = 81)= ', str(round(t[0], 2)), 'p = ', str(round(pval[0], 4)))

Ttest_1sampResult(statistic=array([2.71219225]), pvalue=array([0.00817983]))
 N200 t( df = 81)=  2.71 p =  0.0082


In [18]:
# wilkoxon 
compute_paired_wilcoxon_rank_biserial(oddball_list, control_list, roi, tmin=.2772, tmax=.3202)

Paired Wilcoxon W= 1166.00, p= 0.0200, rank-biserial correlation (r_rb)= -0.2346


{'W': 1166.0,
 'p_value': 0.020024373791233223,
 'rank_biserial_correlation': -0.2345679012345679}

In [7]:
#plot evokeds at 0.9 CI
analysis.processing.getERP(evokeds, roi = ['1Z'], ci = 0.95, invert = True, vlines=[0.0])

In [60]:
import matplotlib.pyplot as plt
roi = ['7Z']
invert = True

contrast = 'Oddball - Control'
fig, ax = plt.subplots()
mne.viz.plot_compare_evokeds({contrast:diff_waves},
                            legend=None, title=contrast,
                             picks=roi, ci =0.8, axes= ax ,vlines = [0.0, 0.3070,0.4177]
                            )
if(invert): 
    ax.invert_yaxis()
    
# ax.set_ylabel("V")
plt.show()

In [81]:
import matplotlib.pyplot as plt
roi = ['1Z']
invert = True

contrast = 'Oddball - Control'
fig, ax = plt.subplots()
mne.viz.plot_compare_evokeds({contrast:diff_waves},
                            legend=None, title=contrast,
                             picks=roi, ci =0.8, axes= ax ,vlines = [0.0, 0.3419]
                            )
if(invert): 
    ax.invert_yaxis()
    
# ax.set_ylabel("V")
plt.show()

In [19]:
import numpy as np
evoked_data = np.array([ [np.mean(np.mean(e.get_data(picks=['1Z'], tmin=.3, tmax=.36), axis=1), axis=0)] for e in diff_waves])
from scipy import stats 

result = stats.ttest_1samp(evoked_data, 0)
t =  result.statistic
print(result)
pval = result.pvalue
# df = result.df
# print('t( df = ', str(round(df[0], 2)) ,')= ', str(round(t[0], 2)), 'p = ', str(round(pval[0], 4)))
print(' N200 t( df = 81)= ', str(round(t[0], 2)), 'p = ', str(round(pval[0], 4)))

Ttest_1sampResult(statistic=array([2.43574959]), pvalue=array([0.01708381]))
 N200 t( df = 81)=  2.44 p =  0.0171


In [34]:
# wilkoxon 
compute_paired_wilcoxon_rank_biserial(oddball_list, control_list, ['1Z'],  tmin=.3, tmax=.36)

Paired Wilcoxon W= 1205.00, p= 0.0322, rank-biserial correlation (r_rb)= -0.1852


{'W': 1205.0,
 'p_value': 0.0321719306124718,
 'rank_biserial_correlation': -0.18518518518518517}

In [8]:
import numpy as np
evooddballs =evokeds['oddball']
evoodcontrol =evokeds['control']
info = evooddballs[0].info

# raw = raw.apply_function(lambda x: x * 1e6)

evo_oddball_data_list_array = np.array([ ob.get_data() * 1e6 for ob in evooddballs])
evo_control_data_list_array = np.array([ ob.get_data() * 1e6 for ob in evoodcontrol])

ob_evoked_array = mne.EvokedArray(
    np.mean(evo_oddball_data_list_array, 0), info, tmin=-0.2, nave=evo_oddball_data_list_array.shape[0], comment="odd balls"
)

control_evoked_array = mne.EvokedArray(
    np.mean(evo_control_data_list_array, 0), info, tmin=-0.2, nave=evo_control_data_list_array.shape[0], comment="odd balls"
)
times = np.arange(-0.05, 0.6, 0.05)


In [12]:
ob_evoked_array.plot_topomap(times, ch_type="eeg", vlim= (-7.5, 7.5))
# Save the figure to a PDF file
plt.savefig('v_o_topo_ob.pdf', format='pdf', bbox_inches='tight')

In [13]:
control_evoked_array.plot_topomap(times, ch_type="eeg", vlim= (-7.5, 7.5))
plt.savefig('v_o_topo_control.pdf', format='pdf', bbox_inches='tight')

In [8]:
import matplotlib.animation as animation 
writervideo = animation.FFMpegWriter(fps=60) 

times = np.arange(-0.05, 0.6, 0.01)
fig, anim = ob_evoked_array.animate_topomap(times=times, ch_type="eeg", frame_rate= 8, blit=False, vmin =-7.5, vmax= 7.5, butterfly= True)
# anim.save('ob_evoked_topo.gif', writer="ffmpeg") 

MovieWriter ffmpeg unavailable; using Pillow instead.


In [9]:
fig, anim = control_evoked_array.animate_topomap(times=times, ch_type="eeg", frame_rate= 8, blit=False, vmin =-7.5, vmax= 7.5, butterfly= True)
# anim.save('control_evoked_topo.gif', writer="ffmpeg") 

MovieWriter ffmpeg unavailable; using Pillow instead.


In [42]:
def plotSpectrogram(chan_list):
        # Pick a time frequency region 
    # shape (5, 2, 96, 163)

    picks = chan_list

    chan_indexes =  [ch_names.index(item) for item in chan_list]


    spectrogram_control = np.mean(np.mean(control_tfr_array[:,chan_indexes, : ,:], axis = 1), axis = 0)
    spectrogram_oddball =  np.mean(np.mean(oddball_tfr_array[:,chan_indexes, : ,:], axis = 1), axis = 0)

    times = 1e3 * tfr_times  # change unit to ms


    fig, (ax, ax2, ax3) = plt.subplots(3, 1)

    ax.imshow(spectrogram_control,
            extent=[times[0], times[-1], tfr_freqs[0], tfr_freqs[-1]],
            aspect='auto', origin='lower')

    ax.set_xlabel('Time (ms)')
    ax.set_ylabel('Frequency (Hz)')
    ax.set_title(f'Spectrogram control '+  str(picks))

    ax2.imshow(spectrogram_oddball,
            extent=[times[0], times[-1], tfr_freqs[0], tfr_freqs[-1]],
            aspect='auto', origin='lower')

    ax2.set_xlabel('Time (ms)')
    ax2.set_ylabel('Frequency (Hz)')
    ax2.set_title(f'Spectrogram odd ' +  str(picks))


    c = ax3.imshow(spectrogram_oddball - spectrogram_control,
            extent=[times[0], times[-1], tfr_freqs[0], tfr_freqs[-1]],
            aspect='auto', origin='lower')


    ax3.set_xlabel('Time (ms)')
    ax3.set_ylabel('Frequency (Hz)')
    ax3.set_title(f'Spectrogram odd - control ' + str(picks))

    # Create a new axis for the colorbar
    cbar_ax = fig.add_axes([0.15, 0.05, 0.7, 0.03])  # [left, bottom, width, height]
    plt.colorbar(c, cax=cbar_ax, orientation='horizontal', label="Relative Power")
   
    # Adjust the spacing between subplots
    plt.subplots_adjust(hspace=0.5)  # hspace can be adjusted for more or less space


    plt.show()

    return spectrogram_control, spectrogram_oddball

In [43]:

def plotRoI(spectrogram_oddball, spectrogram_control, time_roi = [], freq_roi = [], chan_list = []):

    picks = chan_list

    # Pick a time frequency region 
    # shape (5, 2, 96, 163)
    diff = spectrogram_oddball - spectrogram_control
    times = 1e3 * tfr_times  # change unit to ms


    # Create figure and axes
    fig, ax = plt.subplots()

    c =  ax.imshow(diff, extent=[times[0], times[-1], freqs[0], freqs[-1]],
            aspect='auto', origin='lower')

    for index, (time_range, freq_range) in  enumerate(zip(time_roi,freq_roi )):
        rect = patches.Rectangle((time_range[0], freq_range[0]), (time_range[1] - time_range[0]), (freq_range[1] - freq_range[0]), linewidth=1, edgecolor='r', facecolor='none')
        ax.text((time_range[0]+ time_range[1])/2, (freq_range[0]+ freq_range[1])/2, f'ROI {index +1}')
        ax.add_patch(rect)

    plt.colorbar(c, label = "Relative power")


    plt.title('Oddball - Control for "' + str(picks) + '"',  fontweight ="bold")
    plt.show()

In [49]:
# get indexes of the interested regions 
def getIndexArrayOfArray ( main_array, value_list, divisor):
    index_list =[]
    for val in value_list:
        idx = np.argmin( np.abs(main_array - (val / divisor)))
        index_list.append(idx)
    return index_list

In [44]:
def getROIDataFrame ( time1idx, freq1idx, time2idx = None, freq2idx = None, chan_list = [] ):

    chan_indexes =  [ch_names.index(item) for item in chan_list]

    if((time2idx is not None) and (freq2idx is not None)):
        labels = [ 'ROI-1 Control','ROI-1 Odd','ROI-2 Control','ROI-2 Odd' ]
        data = np.zeros((4,oddball_tfr_array.shape[0]));

        # resultMat shape (5, 2, 96, 163)
        # np.mean(resultMat, axis=1)


        data[0,:] =  np.mean(np.mean(np.mean(control_tfr_array[:,chan_indexes, freq1idx[0]:freq1idx[1], time1idx[0]: time1idx[1]], axis = 3), axis = 2), axis =1)
        data[1,:] =  np.mean(np.mean(np.mean(oddball_tfr_array[:,chan_indexes, freq1idx[0]:freq1idx[1], time1idx[0]: time1idx[1]], axis = 3), axis = 2), axis =1)

        data[2,:] =  np.mean(np.mean(np.mean(control_tfr_array[:,chan_indexes, freq2idx[0]:freq2idx[1], time2idx[0]: time2idx[1]], axis = 3), axis = 2), axis =1)
        data[3,:] =  np.mean(np.mean(np.mean(oddball_tfr_array[:,chan_indexes, freq2idx[0]:freq2idx[1], time2idx[0]: time2idx[1]], axis = 3), axis = 2), axis =1)

    else:
        labels = [ 'ROI-1 Control','ROI-1 Odd'];
        data = np.zeros((2,oddball_tfr_array.shape[0]));

        # resultMat shape (5, 2, 96, 163)
        # np.mean(resultMat, axis=1)


        data[0,:] =  np.mean(np.mean(np.mean(control_tfr_array[:,chan_indexes, freq1idx[0]:freq1idx[1], time1idx[0]: time1idx[1]], axis = 3), axis = 2), axis =1)
        data[1,:] =  np.mean(np.mean(np.mean(oddball_tfr_array[:,chan_indexes, freq1idx[0]:freq1idx[1], time1idx[0]: time1idx[1]], axis = 3), axis = 2), axis =1)


    return data, labels

In [45]:
from scipy import stats
import numpy as np

def getRoIstats(data, labels):

    d_mean = np.mean(data, axis= 1)
    d_std = np.std(data, axis= 1)

    x_pos = np.arange(len(labels))

    datai = data

    for col in range(datai.shape[1] -1, -1, -1):
        variance_per_subjecti =  np.abs(datai[:,col] - d_mean)
        dist_from_2stds =  variance_per_subjecti - 2 * d_std
        # print(dist_from_2stds)
        for val in dist_from_2stds:
            if(val> 0):
                datai = np.delete(datai,(col), axis=1)
                # print("deleted")
                break;

    d_mean = np.mean(datai, axis= 1)
    d_std = np.std(datai, axis= 1) 
    n = datai.shape[1]
    yerr = d_std / np.sqrt(n) * stats.t.ppf(1-0.05/2, n - 1)   


    if datai.shape[0] > 2:
        print("T-test for the ROI 1")
        result_roi_1 = stats.ttest_ind(datai[0,:], datai[1,:])
        print("T-statistic: "+ str(result_roi_1.statistic))    
        print("P - Val : "+ str(result_roi_1.pvalue))


        print("T-test for the ROI 2")
        result_roi_1 = stats.ttest_ind(datai[2,:], datai[3,:])
        print("T-statistic: "+ str(result_roi_1.statistic))  
        print("P - Val : "+ str(result_roi_1.pvalue))

    else:
        print("T-test for the ROI 1")
        result_roi_1 = stats.ttest_ind(datai[0,:], datai[1,:])
        print("T-statistic: "+ str(result_roi_1.statistic)) 
        print("P - Val : "+ str(result_roi_1.pvalue))    
        

    # Build the plot
    fig, ax = plt.subplots()
    ax.bar(x_pos, d_mean, yerr=yerr, align='center', alpha=0.5, ecolor='black', capsize=10)
    ax.set_ylabel('Relative power')
    ax.set_xticks(x_pos)
    ax.set_xticklabels(labels)
    ax.set_title('Conditional power for each ROI')
    ax.yaxis.grid(True)

    # Save the figure and show
    plt.tight_layout()
    # plt.savefig('bar_plot_with_error_bars.png')
    plt.show()


In [46]:
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt

def getRoIstats_wilcoxon(data, labels):
    """
    Computes Wilcoxon signed-rank tests and rank-biserial correlation for paired ROIs, 
    along with plotting conditional power for each ROI.
    
    Parameters:
    - data: A 2D numpy array where each row represents an ROI and each column represents a subject's data.
    - labels: A list of labels corresponding to the ROIs.
    
    Returns:
    - None (prints the test results and displays a bar plot)
    """
    # Calculate mean and standard deviation for each ROI
    d_mean = np.mean(data, axis=1)
    d_std = np.std(data, axis=1)

    # Prepare indices for x-axis positions of the bars
    x_pos = np.arange(len(labels))

    # Clean the data by removing outliers (values more than 2 standard deviations away)
    datai = data.copy()
    for col in range(datai.shape[1] - 1, -1, -1):
        variance_per_subjecti = np.abs(datai[:, col] - d_mean)
        dist_from_2stds = variance_per_subjecti - 2 * d_std
        if np.any(dist_from_2stds > 0):
            datai = np.delete(datai, col, axis=1)

    # Recompute mean, std, and standard error after outlier removal
    d_mean = np.mean(datai, axis=1)
    d_std = np.std(datai, axis=1)
    n = datai.shape[1]
    yerr = d_std / np.sqrt(n) * stats.t.ppf(1 - 0.05 / 2, n - 1)

    # Perform Wilcoxon signed-rank tests if we have more than two ROIs
    if datai.shape[0] > 2:
        print("Wilcoxon test for the ROI 1")
        result_roi_1 = stats.wilcoxon(datai[0, :], datai[1, :], zero_method='wilcox', correction=True)
        rank_biserial_1 = (np.sum(datai[0, :] > datai[1, :]) - np.sum(datai[0, :] < datai[1, :])) / n
        print(f"Wilcoxon W= {result_roi_1.statistic:.2f}, p= {result_roi_1.pvalue:.4f}, rank-biserial correlation= {rank_biserial_1:.4f}")

        print("Wilcoxon test for the ROI 2")
        result_roi_2 = stats.wilcoxon(datai[2, :], datai[3, :], zero_method='wilcox', correction=True)
        rank_biserial_2 = (np.sum(datai[2, :] > datai[3, :]) - np.sum(datai[2, :] < datai[3, :])) / n
        print(f"Wilcoxon W= {result_roi_2.statistic:.2f}, p= {result_roi_2.pvalue:.4f}, rank-biserial correlation= {rank_biserial_2:.4f}")
    else:
        print("Wilcoxon test for the ROI 1")
        result_roi_1 = stats.wilcoxon(datai[0, :], datai[1, :], zero_method='wilcox', correction=True)
        rank_biserial_1 = (np.sum(datai[0, :] > datai[1, :]) - np.sum(datai[0, :] < datai[1, :])) / n
        print(f"Wilcoxon W= {result_roi_1.statistic:.2f}, p= {result_roi_1.pvalue:.4f}, rank-biserial correlation= {rank_biserial_1:.4f}")

    # Build the plot
    fig, ax = plt.subplots()
    ax.bar(x_pos, d_mean, yerr=yerr, align='center', alpha=0.5, ecolor='black', capsize=10)
    ax.set_ylabel('Relative power')
    ax.set_xticks(x_pos)
    ax.set_xticklabels(labels)
    ax.set_title('Conditional power for each ROI')
    ax.yaxis.grid(True)

    # Show the plot
    plt.tight_layout()
    plt.show()


In [55]:
chan_list = ['7Z']
spectrogram_control, spectrogram_oddball = plotSpectrogram(chan_list)
time_roi = [[250, 384], [445, 532]]
freq_roi = [[2, 6], [2, 4]] 
plotRoI(spectrogram_oddball, spectrogram_control, time_roi, freq_roi, chan_list)

time1idx = getIndexArrayOfArray(tfr_times, time_roi[0], 1e3)
time2idx = None
if(len(time_roi) > 1): 
    time2idx = getIndexArrayOfArray(tfr_times, time_roi[1], 1e3) 

freq1idx = getIndexArrayOfArray(tfr_freqs, freq_roi[0], 1)
freq2idx = None
if(len(freq_roi) > 1):  
    freq2idx = getIndexArrayOfArray(tfr_freqs, freq_roi[1], 1) 

data, labels = getROIDataFrame (time1idx,freq1idx, time2idx, freq2idx, chan_list )
getRoIstats(data, labels)

T-test for the ROI 1
T-statistic: -6.071594222676712
P - Val : 9.932626946464812e-09
T-test for the ROI 2
T-statistic: -5.245366995197791
P - Val : 5.224430867524854e-07


In [51]:
getRoIstats_wilcoxon(data, labels)

Wilcoxon test for the ROI 1
Wilcoxon W= 439.00, p= 0.0000, rank-biserial correlation= -0.6053
Wilcoxon test for the ROI 2
Wilcoxon W= 528.00, p= 0.0000, rank-biserial correlation= -0.5263


In [56]:
chan_list = ['1Z']
spectrogram_control, spectrogram_oddball = plotSpectrogram(chan_list)
time_roi = [[275, 398]]
freq_roi = [[3.9, 8]] 
plotRoI(spectrogram_oddball, spectrogram_control, time_roi, freq_roi, chan_list)

time1idx = getIndexArrayOfArray(tfr_times, time_roi[0], 1e3)
time2idx = None
if(len(time_roi) > 1): 
    time2idx = getIndexArrayOfArray(tfr_times, time_roi[1], 1e3) 

freq1idx = getIndexArrayOfArray(tfr_freqs, freq_roi[0], 1)
freq2idx = None
if(len(freq_roi) > 1):  
    freq2idx = getIndexArrayOfArray(tfr_freqs, freq_roi[1], 1) 

data, labels = getROIDataFrame (time1idx,freq1idx, time2idx, freq2idx, chan_list )
getRoIstats(data, labels)

T-test for the ROI 1
T-statistic: -5.190994581993983
P - Val : 6.791986709680032e-07


In [53]:
getRoIstats_wilcoxon(data, labels)

Wilcoxon test for the ROI 1
Wilcoxon W= 618.00, p= 0.0000, rank-biserial correlation= -0.3867


In [54]:
# frontal 
chan_list = ['1L', '1R','1Z','1LC', '1LB', '2L', '2R', '1RB', '1RC']
spectrogram_control, spectrogram_oddball = plotSpectrogram(chan_list)
time_roi = [[275, 398]]
freq_roi = [[3.9, 8]] 
plotRoI(spectrogram_oddball, spectrogram_control, time_roi, freq_roi, chan_list)

time1idx = getIndexArrayOfArray(tfr_times, time_roi[0], 1e3)
time2idx = None
if(len(time_roi) > 1): 
    time2idx = getIndexArrayOfArray(tfr_times, time_roi[1], 1e3) 

freq1idx = getIndexArrayOfArray(tfr_freqs, freq_roi[0], 1)
freq2idx = None
if(len(freq_roi) > 1):  
    freq2idx = getIndexArrayOfArray(tfr_freqs, freq_roi[1], 1) 

data, labels = getROIDataFrame (time1idx,freq1idx, time2idx, freq2idx, chan_list )
getRoIstats(data, labels)

T-test for the ROI 1
T-statistic: -5.120239856837816
P - Val : 9.483201885913527e-07


In [16]:
# occipital  
chan_list = ['10L', '9Z','10R', '9L', '8Z', '9R']
spectrogram_control, spectrogram_oddball = plotSpectrogram(chan_list)
time_roi = [[275, 398]]
freq_roi = [[3.9, 8]] 
plotRoI(spectrogram_oddball, spectrogram_control, time_roi, freq_roi, chan_list)

time1idx = getIndexArrayOfArray(tfr_times, time_roi[0], 1e3)
time2idx = None
if(len(time_roi) > 1): 
    time2idx = getIndexArrayOfArray(tfr_times, time_roi[1], 1e3) 

freq1idx = getIndexArrayOfArray(tfr_freqs, freq_roi[0], 1)
freq2idx = None
if(len(freq_roi) > 1):  
    freq2idx = getIndexArrayOfArray(tfr_freqs, freq_roi[1], 1) 

data, labels = getROIDataFrame (time1idx,freq1idx, time2idx, freq2idx, chan_list )
getRoIstats(data, labels)


T-test for the ROI 1
T-statistic: -3.5879671144103233
P - Val : 0.00045054541784070545


In [110]:
# left  
chan_list = ['3LB', '3LC','2LB', '1LA', '2LA', '2LC']
spectrogram_control, spectrogram_oddball = plotSpectrogram(chan_list)
time_roi = [[275, 398]]
freq_roi = [[3.9, 8]] 
plotRoI(spectrogram_oddball, spectrogram_control, time_roi, freq_roi, chan_list)

time1idx = getIndexArrayOfArray(tfr_times, time_roi[0], 1e3)
time2idx = None
if(len(time_roi) > 1): 
    time2idx = getIndexArrayOfArray(tfr_times, time_roi[1], 1e3) 

freq1idx = getIndexArrayOfArray(tfr_freqs, freq_roi[0], 1)
freq2idx = None
if(len(freq_roi) > 1):  
    freq2idx = getIndexArrayOfArray(tfr_freqs, freq_roi[1], 1) 

data, labels = getROIDataFrame (time1idx,freq1idx, time2idx, freq2idx, chan_list )
getRoIstats(data, labels)

T-test for the ROI 1
T-statistic: -3.556623359443203
P - Val : 0.000508897271797458


In [111]:
# right  
chan_list =  ['3RB', '3RC','2RB', '1RA', '2RA', '2RC']
spectrogram_control, spectrogram_oddball = plotSpectrogram(chan_list)
time_roi = [[275, 398]]
freq_roi = [[3.9, 8]] 
plotRoI(spectrogram_oddball, spectrogram_control, time_roi, freq_roi, chan_list)

time1idx = getIndexArrayOfArray(tfr_times, time_roi[0], 1e3)
time2idx = None
if(len(time_roi) > 1): 
    time2idx = getIndexArrayOfArray(tfr_times, time_roi[1], 1e3) 

freq1idx = getIndexArrayOfArray(tfr_freqs, freq_roi[0], 1)
freq2idx = None
if(len(freq_roi) > 1):  
    freq2idx = getIndexArrayOfArray(tfr_freqs, freq_roi[1], 1) 

data, labels = getROIDataFrame (time1idx,freq1idx, time2idx, freq2idx, chan_list )
getRoIstats(data, labels)

T-test for the ROI 1
T-statistic: -3.9863366445905717
P - Val : 0.0001050429047102989
